In [1]:
# This jupyter-notebook is for my study. Codes was modified based on the following references
# https://github.com/keras-team/keras/blob/master/examples/lstm_seq2seq.py
# https://towardsdatascience.com/seq2seq-model-in-tensorflow-ec0c557e560f

# Set up environment

In [2]:
import os

# make only specific GPU to be utilized
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

# select GPU to run on
GPU = "0"
os.environ["CUDA_VISIBLE_DEVICES"] = GPU

# stop GPU
# os.environ["CUDA_VISIBLE_DEVICES"] ="-1"

# set GPU to be deterministic 
os.environ["PYTHONHASHSEED"] = "0" # set hash environment
os.environ["TF_CUDNN_USE_AUTOTUNE"] = "0" # use cuDNN function to retrieve the best algorithm
os.environ["TF_CUDNN_CONVOLUTION_BWD_FILTER_ALGO_DETERMINISTIC"]='1' # use cuDNN deterministic algorithms
os.environ['TF_DETERMINISTIC_OPS'] = '1' # deterministic setting

# set numpy, python, tensorflow random seed
from numpy.random import seed
import random
random.seed(10)
seed(10)
from tensorflow import set_random_seed
set_random_seed(20)

# GPU memory control
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
# config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.40
sess = tf.Session(config=config)

# Set up data path and parameters

In [3]:
# dataset was download from http://www.manythings.org/anki/

In [4]:
# Set up path to the data textfile
data_path = 'data/jpn-eng/jpn.txt'
# data_path = 'data/tha-eng/tha.txt'
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
print ("total line", len(lines))

total line 45449


In [5]:
# Set up parameters for training
batch_size = 64  # Batch size for training.
epochs = 200  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
start_samples1 = 0
end_samples1 = 5000 
start_samples2 = 40000
end_samples2 = 45000 

# Process data

In [6]:
from __future__ import print_function
 
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np
 
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

for line in lines[start_samples1: min(end_samples1, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

for line in lines[start_samples2: min(end_samples2, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

# Set data length
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])
 
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 75
Number of unique output tokens: 1921
Max sequence length for inputs: 79
Max sequence length for outputs: 55


Using TensorFlow backend.


In [7]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    print (i, input_texts, target_texts)
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# Model

In [8]:
print (num_encoder_tokens, encoder_input_data.shape)
print (num_decoder_tokens, decoder_input_data.shape)

75 (10000, 79, 75)
1921 (10000, 55, 1921)


In [9]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 75)     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 1921)   0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 256), (None, 339968      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 256),  2230272     input_2[0][0]                    
                                                                 lstm_1[0][1]               

In [10]:
print ([encoder_input_data.shape, decoder_input_data.shape])
print (decoder_target_data.shape)

[(10000, 79, 75), (10000, 55, 1921)]
(10000, 55, 1921)


In [11]:
import time

# Run training
start = time.time()

model_history = model.fit([encoder_input_data, decoder_input_data], 
                          decoder_target_data,
                          batch_size=batch_size,
                          epochs=epochs,
                          validation_split=0.2)
end = time.time()
print ("calculation time: ", end - start)

# # Save model
# model.save('seq2seq.h5')

W1222 10:43:59.581205 140277091657536 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W1222 10:44:01.042556 140277091657536 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



Train on 8000 samples, validate on 2000 samples
Epoch 1/200
8000/8000 [==============================] - 18s 2ms/step - loss: 1.7339 - accuracy: 0.7307 - val_loss: 2.8443 - val_accuracy: 0.5060
Epoch 2/200
8000/8000 [==============================] - 15s 2ms/step - loss: 1.3725 - accuracy: 0.7530 - val_loss: 2.5601 - val_accuracy: 0.5590
Epoch 3/200
8000/8000 [==============================] - 15s 2ms/step - loss: 1.2223 - accuracy: 0.7810 - val_loss: 2.3946 - val_accuracy: 0.5640
Epoch 4/200
8000/8000 [==============================] - 16s 2ms/step - loss: 1.1326 - accuracy: 0.7908 - val_loss: 2.2378 - val_accuracy: 0.5918
Epoch 5/200
8000/8000 [==============================] - 15s 2ms/step - loss: 1.0202 - accuracy: 0.8072 - val_loss: 2.1438 - val_accuracy: 0.6084
Epoch 6/200
8000/8000 [==============================] - 15s 2ms/step - loss: 0.9561 - accuracy: 0.8151 - val_loss: 2.0870 - val_accuracy: 0.6125
Epoch 7/200
8000/8000 [==============================] - 15s 2ms/step - loss

8000/8000 [==============================] - 15s 2ms/step - loss: 0.2710 - accuracy: 0.9441 - val_loss: 2.3554 - val_accuracy: 0.6446
Epoch 57/200
8000/8000 [==============================] - 15s 2ms/step - loss: 0.2656 - accuracy: 0.9454 - val_loss: 2.3651 - val_accuracy: 0.6432
Epoch 58/200
8000/8000 [==============================] - 15s 2ms/step - loss: 0.2617 - accuracy: 0.9464 - val_loss: 2.3863 - val_accuracy: 0.6428
Epoch 59/200
8000/8000 [==============================] - 15s 2ms/step - loss: 0.2566 - accuracy: 0.9473 - val_loss: 2.4010 - val_accuracy: 0.6426
Epoch 60/200
8000/8000 [==============================] - 15s 2ms/step - loss: 0.2527 - accuracy: 0.9483 - val_loss: 2.3944 - val_accuracy: 0.6436
Epoch 61/200
8000/8000 [==============================] - 15s 2ms/step - loss: 0.2482 - accuracy: 0.9492 - val_loss: 2.4312 - val_accuracy: 0.6434
Epoch 62/200
8000/8000 [==============================] - 15s 2ms/step - loss: 0.2445 - accuracy: 0.9503 - val_loss: 2.4158 - val_a

Epoch 112/200
8000/8000 [==============================] - 15s 2ms/step - loss: 0.1475 - accuracy: 0.9688 - val_loss: 2.9409 - val_accuracy: 0.6385
Epoch 113/200
8000/8000 [==============================] - 15s 2ms/step - loss: 0.1466 - accuracy: 0.9689 - val_loss: 2.9408 - val_accuracy: 0.6395
Epoch 114/200
8000/8000 [==============================] - 15s 2ms/step - loss: 0.1454 - accuracy: 0.9692 - val_loss: 2.9333 - val_accuracy: 0.6399
Epoch 115/200
8000/8000 [==============================] - 15s 2ms/step - loss: 0.1439 - accuracy: 0.9696 - val_loss: 2.9424 - val_accuracy: 0.6390
Epoch 116/200
8000/8000 [==============================] - 15s 2ms/step - loss: 0.1429 - accuracy: 0.9696 - val_loss: 2.9591 - val_accuracy: 0.6378
Epoch 117/200
8000/8000 [==============================] - 15s 2ms/step - loss: 0.1415 - accuracy: 0.9702 - val_loss: 2.9596 - val_accuracy: 0.6399
Epoch 118/200
8000/8000 [==============================] - 15s 2ms/step - loss: 0.1405 - accuracy: 0.9702 - val_

8000/8000 [==============================] - 14s 2ms/step - loss: 0.1027 - accuracy: 0.9773 - val_loss: 3.2035 - val_accuracy: 0.6388
Epoch 168/200
8000/8000 [==============================] - 15s 2ms/step - loss: 0.1018 - accuracy: 0.9774 - val_loss: 3.2088 - val_accuracy: 0.6397
Epoch 169/200
8000/8000 [==============================] - 15s 2ms/step - loss: 0.1013 - accuracy: 0.9775 - val_loss: 3.2056 - val_accuracy: 0.6389
Epoch 170/200
8000/8000 [==============================] - 15s 2ms/step - loss: 0.1006 - accuracy: 0.9775 - val_loss: 3.2127 - val_accuracy: 0.6391
Epoch 171/200
8000/8000 [==============================] - 14s 2ms/step - loss: 0.0996 - accuracy: 0.9776 - val_loss: 3.2249 - val_accuracy: 0.6401
Epoch 172/200
8000/8000 [==============================] - 14s 2ms/step - loss: 0.0986 - accuracy: 0.9781 - val_loss: 3.2310 - val_accuracy: 0.6383
Epoch 173/200
8000/8000 [==============================] - 16s 2ms/step - loss: 0.0983 - accuracy: 0.9780 - val_loss: 3.2395 -

In [12]:
import matplotlib.pyplot as plt
def plot_accuracy(history):
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    t = f.suptitle('Seq2Seq Performance', fontsize=12)
    f.subplots_adjust(top=0.85, wspace=0.3)

    epoch_lenght = len(history.history['accuracy'])+1
    epoch_list = list(range(1,epoch_lenght))
    
    # Plot accuracy
    ax1.plot(epoch_list, history.history['accuracy'], label='Train Accuracy')
    ax1.plot(epoch_list, history.history['val_accuracy'], label='Validation Accuracy')
    ax1.set_xticks(np.arange(0, epoch_lenght, 20))
    ax1.set_ylabel('Accuracy Value')
    ax1.set_xlabel('Epoch')
    ax1.set_title('Accuracy')
    l1 = ax1.legend(loc="best")

    # Plot loss
    ax2.plot(epoch_list, history.history['loss'], label='Train Loss')
    ax2.plot(epoch_list, history.history['val_loss'], label='Validation Loss')
    ax2.set_xticks(np.arange(0, epoch_lenght, 20))
    ax2.set_ylabel('Loss Value')
    ax2.set_xlabel('Epoch')
    ax2.set_title('Loss')
    l2 = ax2.legend(loc="best")
plot_accuracy(model_history)

In [13]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [14]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [15]:
n = 1
for seq_index in range(10):
    input_seq = encoder_input_data[n + seq_index: n + seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(input_texts[n + seq_index], '-', decoded_sentence)

Go. - 行きなさい。

Hi. - こんにちは！

Hi. - こんにちは！

Hi. - こんにちは！

Run. - 走れ。

Run. - 走れ。

Who? - 誰？

Wow! - すごい！

Wow! - すごい！

Wow! - すごい！



# Test a translated sentence

In [16]:
def translate_word(input_sentence):
    input_index = input_texts.index(input_sentence)
    input_seq = encoder_input_data[input_index:input_index+1]
    decoded_sentence = decode_sequence(input_seq)
    print(input_texts[input_index], '->', decoded_sentence)

In [17]:
translate_word("Jump!")

Jump! -> 飛び跳ねて！



In [18]:
translate_word("Run.")

Run. -> 走れ。



In [19]:
translate_word("Close your books.")

Close your books. -> 本を閉じなさい。



In [20]:
translate_word("Come and help me.")

Come and help me. -> 私といっしょに来なさい。



In [21]:
translate_word("Life is too short to worry about things like that.")

Life is too short to worry about things like that. -> そんなことをしたら逆効果になる。



In [22]:
translate_word("Tom said that it probably wouldn't take too much time.")

Tom said that it probably wouldn't take too much time. -> トムは何か忘れた気がすると言っていた。

